In [16]:
from importlib import reload
if 'andor' in dir():
    reload(andor)
else:
    import andor
#
from andorCodes import AndorCodes
from ipywidgets import widgets
from IPython.display import clear_output
import time
import asyncio
import matplotlib
import numpy
matplotlib.use('agg')
from matplotlib import pyplot

In [11]:
## Initialise the SDK and the asyncio events
#

readout=asyncio.Event() # when set, the readout can commence
readout.clear()
abort=asyncio.Event() # when set, the readout can stop
abort.clear()

try:
    andor.initialize_ctypes()
except:
    raise RuntimeError("Cannot load shared libray, not installed?")
try:
    andor.initialize_sdk()
except:
    raise RuntimeError("Cannot initialise SDK, already initialised elsewhere?")

print("Initialised SDK")
andor.choose_camera(0) # first
print("Set current camera to be #0 (first)")
ccd_height, ccd_width=andor.detector_size()
print(f"Size of detector is {ccd_height}x{ccd_width}")

#executor=concurrent.futures.ThreadPoolExecutor(max_workers=3)

RuntimeError: Cannot initialise SDK, already initialised elsewhere?

In [20]:
## detector configuration parameters
#
hbinning,vbinning=(1,1) # horizontal then vertical
region_lrbt=1,1024,1,1024#100,200,100,200
region=numpy.empty([1024,1024])
region=andor.Region(hbinning,vbinning,region_lrbt[0],region_lrbt[1],region_lrbt[2],region_lrbt[3])
exposure_time=0.05 # [s]
ttl_high=True
shutter_mode='Open'
open_time,close_time=0,0
trigger_mode='Internal'
acq_mode='Continuous'
read_mode='Image'

RuntimeError: SDK not initialized

In [3]:
andor.set_region(region)
andor.set_exposure(exposure_time) ; print(f"Asked for exposure time is {1e3*exposure_time:.1f}ms")
andor.set_shutter(shutter_mode,ttl_high,open_time,close_time)
andor.set_trigger(trigger_mode) ; print(f"Tigger mode is '{trigger_mode}'")
# No need to do this, the default it seems
andor.set_acquisition(acq_mode) ; print(f"Acquisition mode is '{acq_mode}'")
andor.set_read(read_mode) ; print(f"Read mode is '{read_mode}'")
print(f"Binning HxV is {hbinning}x{vbinning}")
print("ROI is (L,R,B,T)={0[0]}x{0[1]},{0[2]},{0[3]}".format(region_lrbt))
times=andor.get_timing()
print(f"Therefore, exposure time is {1e3*times[0]:.1f}ms and acquisition time is {1e3*times[1]:.1f}ms")
img_capacity=andor.get_size_of_circular_buffer()
print(f"Expect {times[1]*img_capacity:.2f}s before circular buffer is full")

print(f"Shutter {'is' if andor.get_shutter_info() else 'is not'} available")
print(f"Shutter set to TTL o/p {'high' if ttl_high else 'low'}")
print(f"Shutter mode is '{shutter_mode}'")
print(f"Shutter open/close time is {open_time}/{close_time}ms")

NameError: name 'region' is not defined

In [4]:
def abort_and_clear():
    try:
        andor.abort_acquisition()
    except:
        print("No acquisition to abort")
    andor.free_internal_memory()
    return

async def wait_for_ready():
    while andor.get_status()==AndorCodes.DRV_ACQUIRING:
        await asyncio.sleep(0.2)

In [22]:
statusOpW=widgets.Output(
        disabled=False,
        layout=widgets.Layout(
            border='solid 1px blue',
            width='50%',height='400px')
    )
imageOpW=widgets.Output(
        disabled=False,
        layout=widgets.Layout(
            border='solid 1px red',
            margin='0px 10px 10px 0px',
            padding='5px 5px 5px 5px',
            width='50%',height='400px')
    )

startW,stopW,abortW=[
        widgets.Button(description=tLabel,disabled=False,button_style="")
        for tLabel in ("Start","Stop","Abort")
    ]
buttonsHBW=widgets.HBox(
        [startW,stopW,abortW],
        layout=widgets.Layout(
            width='100%',
            border='solid 1px gray',
            margin='0px 10px 10px 0px',
            padding='5px 5px 5px 5px',)
    )
opHBW=widgets.HBox([imageOpW,statusOpW],layout=widgets.Layout(width='100%'))
panelVBW=widgets.VBox([buttonsHBW,opHBW])

@statusOpW.capture(clear_output=True)
def startReadout(button):
    andor.start_acquisition()
    readout.set()
@statusOpW.capture(clear_output=True)
def stopReadout(button):
    abort_and_clear()
    readout.clear()
@statusOpW.capture(clear_output=True)
def abortReadout(button):
    abort_and_clear()
    abort.set()

startW.on_click(startReadout)
stopW.on_click(stopReadout)
abortW.on_click(abortReadout)

In [23]:
async def produce(queue):
    start_time=time.time()
    while not abort.is_set():
        await readout.wait()
        await loop.run_in_executor(None,andor.wait_for_acquisition)
        n_imgs=andor.get_total_number_images_acquired()
#        print(f"\t{time.time()-start_time:.1f}s:# images={n_imgs:3d}\n")
        if andor.last_ret_code==AndorCodes.DRV_NO_NEW_DATA:
            abort.set()
            andor.abort_acquisition()
            print(f"Acquisition ended without data {AndorCodes.values[andor.last_ret_code]}\n")
            break
#                 print(f"# images acquired={andor.get_total_number_images_acquired()}")
        image_indices=andor.get_number_available_images()
#                 opW.value+="# images available={1} ({0[0]}/{0[1]})".format(image_indices,image_indices[1]-image_indices[0]+1)
        ## \/ TODO should be inside a coroutine or farmed out to a thread so the
        ## 
        data,first,last=andor.get_images(image_indices[0],image_indices[1])
        await queue.put([image_indices,time.time()-start_time,data])

    await wait_for_ready()
    await queue.put(None)
#    print('<end>')


async def consume(queue,imagequeue):
    f=0
    while True:
        # wait for an item from the producer
        item = await queue.get()
        f+=1
        if item is None:
            # the producer emits None to indicate that it is done
            break

        # process the item
        data,first,last=item
        data=data.reshape([last-first+1]+list(region.shape))
        print('image indices {0[0]}--{0[1]}'.format(indices))
        imagequeue.put([data,f]) # now shove onto another queue
    print('{end}')

async def imagedisplay(queue):
    with statusOpW:
        starttime=time.time()
        fig=pyplot.figure(figsize=(8,8))
        ax=fig.subplots(1)
        blank=numpy.empty(region.shape,'f')
        img=ax.imshow(blank,vmin=-1,vmax=1)    
        while True: # this never exits
            item = await queue.get()
            clear_output(wait=True)
            # process the item
            data,metadata=item
            img.set_array(data[-1])
            ax.set_title(f"{metadata+data.shape[0]}:{time.time()-starttime:.2f}")
            display(fig)
        print('{end}')

display(panelVBW)
loop = asyncio.get_event_loop()
dataqueue = asyncio.Queue(loop=loop)
imagequeue = asyncio.Queue(loop=loop)
producer = produce(dataqueue)
consumer = consume(dataqueue,imagequeue)
imagedisplayer = imagedisplay(imagequeue)
future_producer = asyncio.ensure_future(producer)
future_consumer = asyncio.ensure_future(consumer)
future_imagedisplayer = asyncio.ensure_future(imagedisplayer)

In [ ]:
status=andor.get_status()
print(f"Status is: {status}={AndorCodes.values[status]}")

---

In [ ]:
abort_and_clear()
andor.shutdown()